In [ ]:
!pip install --upgrade pip setuptools wheel
!apt update && apt install -y ffmpeg libcairo2-dev libpango1.0-dev
!pip install manim
!pip install -q smolagents==1.7.0
!pip install markdownify datasets sentence-transformers duckduckgo-search spaces gradio-tools langchain langchain-community langchain-huggingface faiss-cpu --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates I

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 120.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 108.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 119.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
manim 0.19.0 requires numpy>

In [3]:

from smolagents import CodeAgent, ToolCallingAgent, ManagedAgent, DuckDuckGoSearchTool, VisitWebpageTool
from smolagents import HfApiModel


In [ ]:
import os
import re
import subprocess
import tempfile
from contextlib import contextmanager
from typing import Optional
import logging
from fastapi import HTTPException

In [ ]:
# logging.basicConfig(level=logging.DEBUG)
# logger = logging.getLogger(__name__)


class ManimProcessor:
    @contextmanager
    def create_temp_dir(self):
        temp_dir = tempfile.mkdtemp()
        try:
            yield temp_dir
        finally:
            if os.path.exists(temp_dir):
                for root, dirs, files in os.walk(temp_dir, topdown=False):
                    for name in files:
                        os.remove(os.path.join(root, name))
                    for name in dirs:
                        os.rmdir(os.path.join(root, name))
                os.rmdir(temp_dir)

    def extract_code(self, rag_response: str) -> Optional[str]:
        pattern = r"```python\n(.*?)```"
        match = re.search(pattern, rag_response, re.DOTALL)
        return match.group(1).strip() if match else None

    def save_code(self, code: str, temp_dir: str) -> str:
        scene_file = os.path.join(temp_dir, "scene.py")
        with open(scene_file, "w") as f:
            f.write("from manim import *\n\n")
            f.write(code)
        return scene_file

    def render_scene(
        self, scene_file: str, scene_name: str, temp_dir: str
    ) -> Optional[str]:
        cmd = [
            "python",
            "-m",
            "manim",
            "-pql",
            "--media_dir",
            temp_dir,
            scene_file,
            scene_name,
        ]

        try:
            subprocess.run(cmd, check=True, capture_output=True, text=True)
            video_path = os.path.join(
                temp_dir, "videos", "scene", "480p15", f"{scene_name}.mp4"
            )

            if not os.path.exists(video_path):
                return None

            temp_video = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
            with open(video_path, "rb") as f:
                temp_video.write(f.read())
            return temp_video.name

        except subprocess.CalledProcessError as e:
            raise HTTPException(status_code=500, detail=f"Render error: {e.stderr}")


In [ ]:
from smolagents import HfApiModel, CodeAgent, ManagedAgent
import subprocess
import tempfile
import os
from pathlib import Path
import shutil
import sys
from contextlib import contextmanager
import re

In [ ]:
def extract_code(response):
    """Extract code from model response"""
    code_lines = []
    in_code_block = False
    for line in response.split('\n'):
        if line.strip().startswith('```'):
            in_code_block = not in_code_block
            continue
        if in_code_block:
            code_lines.append(line)
    return '\n'.join(code_lines)

# def _extract_scene_name(self, code):
#     """Extract the Scene class name from the code"""
#     import re
#     # Look for class definitions that end with 'Scene'
#     pattern = r"class\s+(\w+Scene)\s*\("
#     matches = re.findall(pattern, code)
#     return matches[-1] if matches else None

# @contextmanager
# def _create_temp_dir(self):
#     """Context manager for creating and cleaning up temporary directory"""
#     temp_dir = tempfile.mkdtemp()
#     try:
#         yield temp_dir
#     finally:
#         if os.path.exists(temp_dir):
#             shutil.rmtree(temp_dir, ignore_errors=True)
            

In [ ]:
class DependencyManager:
    """Manages dependencies required for Manim"""
    
    @staticmethod
    def install_dependencies():
        """Install required dependencies with specific versions"""
        dependencies = [
            "numpy",
            "scipy",
            "manim",
            "pillow",
            "pycairo",
            "manimpango",
            "networkx",
            "decorator",
            "sympy"
        ]
        
        for dep in dependencies:
            try:
                subprocess.check_call([
                    sys.executable,
                    "-m",
                    "pip",
                    "install",
                    "--no-cache-dir",
                    dep
                ])
            except subprocess.CalledProcessError as e:
                print(f"Error installing {dep}: {e}")
                return False
        return True

In [ ]:
class ManimCodeWriterAgent(ManagedAgent):
    """Agent responsible for converting educational concepts into Manim code"""
    
    def __init__(self, model):
        self.template = """
        You are an expert Manim programmer. Convert the following concept into production-ready Manim code:
        
        CONCEPT: {concept}
        
        REQUIREMENTS:
        1. Create an educational animation under 100 seconds
        2. Use ManimCE syntax
        3. Include proper documentation and comments throughout the code
        4. Use meaningful variable names and follow PEP 8 style guidelines
        5. Implement smooth transitions and animations with appropriate timing
        6. Keep the scenes visually clear and avoid cluttering the screen
        7. Use color schemes that are easy on the eyes and accessible
        8. Include a proper title and labels where appropriate
        
        CODE STRUCTURE:
        - Start with: from manim import *
        - Create ONE main Scene class that ends with 'Scene' (e.g., BinarySearchScene)
        - Break complex animations into separate methods for better readability
        - Initialize all necessary objects and configurations in the constructor
        - Handle edge cases and potential errors gracefully
        
        OPTIMIZATION GUIDELINES:
        - Use appropriate Manim methods rather than reinventing functionality
        - Leverage existing Manim objects and animations when possible
        - Minimize redundant code and hardcoded values
        - Use numpy for efficient mathematical operations
        - Cache calculations when appropriate to avoid redundant computation
        
        Return ONLY the complete, runnable Python code without any explanation.
        """
        
        super().__init__(
            agent=CodeAgent(
                tools=[],
                model=model,
                additional_authorized_imports=["manim", "numpy", "math"]
            ),
            name="manim_writer",
            description="Writes Manim code for educational animations"
        )
    
    def generate_code(self, concept):
        """Generate Manim code for the given concept"""
        prompt = self.template.format(concept=concept)
        response = self.agent.run(prompt)
        return extract_code(response)

In [ ]:
class ManimExecutorAgent(ManagedAgent):
    """Agent responsible for executing and debugging Manim code"""
    def __init__(self, model):
        self.debug_template = """
        You are an expert Manim debugger. Fix the following code that produced an error:
        
        ORIGINAL CODE:
        ```python
        {code}
        ```
        
        ERROR MESSAGE:
        ```
        {error}
        ```
        
        DEBUGGING INSTRUCTIONS:
        1. Carefully analyze the error message and identify the root cause
        2. Look for common Manim pitfalls like:
           - Incorrect object creation or parameters
           - Animation timing issues
           - Missing or invalid attributes
           - Version compatibility problems
           - Resource handling errors
        3. Make the minimum changes needed to fix the issue
        4. Preserve the original functionality and intent of the code
        5. Follow Manim best practices in your fixes
        6. Ensure the code remains compatible with ManimCE version 0.17.3
        
        Return ONLY the complete, corrected code without explanations or markdown formatting. The code should be ready to run without any modifications.
        """
        
        self.manim_processor = ManimProcessor()
        
        super().__init__(
            agent=CodeAgent(
                tools=[],
                model=model,
                additional_authorized_imports=["subprocess", "sys", "os"]
            ),
            name="manim_executor",
            description="Executes and debugs Manim code"
        )
    
    def _extract_scene_name(self, code):
        """Extract the Scene class name from the code"""
        import re
        pattern = r"class\s+(\w+Scene)\s*\("
        matches = re.findall(pattern, code)
        return matches[-1] if matches else None
    
    def debug_code(self, code, error):
        """Debug the code using the model"""
        prompt = self.debug_template.format(code=code, error=error)
        response = self.agent.run(prompt)
        return self.manim_processor.extract_code(response)
    
    def execute_code(self, code):
        """Execute the Manim code and return the path to the generated video"""
        with self.manim_processor.create_temp_dir() as temp_dir:
            try:
                scene_name = self._extract_scene_name(code)
                if not scene_name:
                    raise Exception("Could not find a valid Scene class in the code")
                
                scene_file = self.manim_processor.save_code(code, temp_dir)
                video_path = self.manim_processor.render_scene(scene_file, scene_name, temp_dir)
                
                if video_path:
                    import shutil
                    kaggle_output_path = f"/kaggle/working/{scene_name}.mp4"
                    shutil.copy(video_path, kaggle_output_path)
                    print(f"Video saved to {kaggle_output_path}.")
                    return kaggle_output_path
                else:
                    raise Exception("Video file not found after rendering")
            
            except Exception as e:
                print(f"Error during execution: {str(e)}")
                return self.debug_code(code, str(e))

In [ ]:
class ManimManagerAgent(ManagedAgent):
    """Manager agent that coordinates the animation creation process"""
    
    def __init__(self, model):
        if not DependencyManager.install_dependencies():
            raise RuntimeError("Failed to install required dependencies")
            
        self.writer = ManimCodeWriterAgent(model)
        self.executor = ManimExecutorAgent(model)
        
        super().__init__(
            agent=CodeAgent(
                tools=[],
                model=model,
                managed_agents=[self.writer, self.executor]
            ),
            name="manim_manager",
            description="Manages the creation of educational animations using Manim"
        )
    
    def create_animation(self, concept):
        """Coordinate the creation of a Manim animation for the given concept"""
        code = self.writer.generate_code(concept)
        video_path = self.executor.execute_code(code)
        
        return video_path

In [ ]:
model = HfApiModel("Qwen/Qwen2.5-72B-Instruct")
manager = ManimManagerAgent(model)

video_path = manager.create_animation(
    "Explain how a binary search algorithm works with a sorted array"
)
print(f"Animation created at: {video_path}")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an expert Manim programmer. Convert the following concept into production-ready Manim code:             │
│                                                                                                                 │
│         CONCEPT: Explain how a binary search algorithm works with a sorted array                                │
│                                                                                                                 │
│         REQUIREMENTS:                                                                                           │
│         1. Create an educational animation under 100 seconds                                                    │
│         2. Use ManimCE syntax                                                                                   │
│         3. Include proper documentation and comments throughout the code                                        │
│         4. Use meaningful variable names and follow PEP 8 style guidelines                                      │
│         5. Implement smooth transitions and animations with appropriate timing                                  │
│         6. Keep the scenes visually clear and avoid cluttering the screen                                       │
│         7. Use color schemes that are easy on the eyes and accessible                                           │
│         8. Include a proper title and labels where appropriate                                                  │
│                                                                                                                 │
│         CODE STRUCTURE:                                                                                         │
│         - Start with: from manim import *                                                                       │
│         - Create ONE main Scene class that ends with 'Scene' (e.g., BinarySearchScene)                          │
│         - Break complex animations into separate methods for better readability                                 │
│         - Initialize all necessary objects and configurations in the constructor                                │
│         - Handle edge cases and potential errors gracefully                                                     │
│                                                                                                                 │
│         OPTIMIZATION GUIDELINES:                                                                                │
│         - Use appropriate Manim methods rather than reinventing functionality                                   │
│         - Leverage existing Manim objects and animations when possible                                          │
│         - Minimize redundant code and hardcoded values                                                          │
│         - Use numpy for efficient mathematical operations                                                       │
│         - Cache calculations when appropriate to avoid redundant computation                                    │
│                                                                                                                 │
│         Return ONLY the complete, runnable Python code without any explanation.                                 │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  from manim import *                                                                                              
                                                                                                                   
  class BinarySearchScene(Scene):                                                                                  
      def __init__(self, **kwargs):                                                                                
          super().__init__(**kwargs)                                                                               
          self.array = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]                                                         
          self.target = 11                                                                                         
          self.array_labels = VGroup()                                                                             
          self.initialize_elements()                                                                               
                                                                                                                   
      def initialize_elements(self):                                                                               
          for i, value in enumerate(self.array):                                                                   
              label = Text(str(value), font="Consolas", font_size=36).shift(DOWN * 2 + LEFT * (5 - i) * 1)         
              self.array_labels.add(label)                                                                         
          self.target_label = Text(f"Target: {self.target}", font="Consolas", font_size=36).to_corner(UP + LEFT)   
                                                                                                                   
      def construct(self):                                                                                         
          self.add(self.array_labels, self.target_label)                                                           
          self.wait(1)                                                                                             
          self.explain_binary_search()                                                                             
                                                                                                                   
      def explain_binary_search(self):                                                                             
          low, high = 0, len(self.array) - 1                                                                       
          while low <= high:                                                                                       
              mid = (low + high) // 2                                                                              
              mid_label = self.array_labels[mid]                                                                   
              self.highlight_element(mid_label,[38;2;248;248;242;48;2;39;40;3

Code execution failed at line 'from manim import *' due to: AttributeError:module 'scipy.interpolate._interpolate' 
has no attribute 'RegularGridInterpolator'

[Step 0: Duration 21.56 seconds| Input tokens: 2,315 | Output tokens: 545]

...

Code execution failed at line 'from manim import *' due to: AttributeError:module 'scipy.interpolate._interpolate' 
has no attribute 'RegularGridInterpolator'

[Step 5: Duration 26.85 seconds| Input tokens: 33,358 | Output tokens: 3,762]

Reached max steps.

[Step 6: Duration 26.85 seconds| Input tokens: 42,010 | Output tokens: 4,339]

Video saved to /kaggle/working/BinarySearchScene.mp4.
Animation created at: /kaggle/working/BinarySearchScene.mp4
